In [25]:
from transformers import BasicTokenizer, EarlyStoppingCallback, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

In [26]:
base.reset_seed()

In [27]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [28]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [29]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [30]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_fine", f"~/data/{DATASET}/test-logits_fine", f"~/data/{DATASET}/train-logits-augmented_fine"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [31]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [32]:
vocab = base.get_vocab(all_data_tokens)

In [33]:
word_index = dict(zip(vocab, range(len(vocab))))

In [34]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [35]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


In [36]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


In [37]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [38]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

In [39]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [40]:
num_epochs = 30
batch_size = 128

In [41]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [42]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [43]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [44]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=50, freeze_embed=False)

In [45]:
base.reset_seed()

In [46]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

In [47]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

In [48]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-embedd",
    n_trials=150
)

[I 2025-03-15 09:38:56,798] A new study created in memory with name: Base-embedd


Trial 0 with params: {'learning_rate': 0.0001025350969016849, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 2}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.766200,3.467665,0.176902,0.003538,0.020000,0.006012
2,3.272000,3.152797,0.176902,0.003538,0.020000,0.006012
3,3.119000,3.034884,0.180568,0.033554,0.021365,0.008638
4,2.954000,2.861430,0.310724,0.038926,0.058350,0.040288
5,2.819800,2.703281,0.377635,0.044616,0.075895,0.051160


[I 2025-03-15 09:39:23,379] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 2.6368755339723032e-05, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 4}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.878600,2.127618,0.461045,0.113446,0.122024,0.106041
2,1.680200,1.417544,0.649863,0.307873,0.289397,0.276222
3,1.001300,1.183799,0.712191,0.413680,0.393680,0.384346
4,0.547600,1.117893,0.738772,0.510903,0.480885,0.480604
5,0.254900,1.175429,0.776352,0.644997,0.591157,0.601605
6,0.123700,1.247634,0.765353,0.648757,0.591919,0.597088
7,0.055300,1.434878,0.779102,0.665201,0.630377,0.631732
8,0.025000,1.460705,0.780935,0.688188,0.634103,0.645460
9,0.011600,1.529630,0.772686,0.665478,0.672656,0.649388
10,0.005800,1.483609,0.779102,0.658384,0.647598,0.633620


[I 2025-03-15 13:57:36,579] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.003869196915802795, 'weight_decay': 0.004, 'adam_beta1': 0.92, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.711900,2.029690,0.484876,0.127003,0.143698,0.121388
2,1.595500,1.359704,0.648946,0.285585,0.296562,0.275373
3,0.878000,1.085870,0.735105,0.450182,0.429813,0.428470
4,0.411400,1.183112,0.723190,0.561487,0.483697,0.498100
5,0.194200,1.166628,0.771769,0.632997,0.593689,0.589260
6,0.072300,1.338467,0.780018,0.698036,0.662990,0.658903
7,0.039400,1.347932,0.773602,0.644865,0.663434,0.642289
8,0.019500,1.378293,0.791017,0.683047,0.664363,0.655907
9,0.006000,1.458098,0.787351,0.701418,0.645802,0.654225
10,0.002900,1.443788,0.789184,0.724586,0.686557,0.682209


[I 2025-03-15 13:59:12,383] Trial 149 pruned. 


In [49]:
print(best_trial)

BestRun(run_id='52', objective=0.7267722601596618, hyperparameters={'learning_rate': 0.004185238693319757, 'weight_decay': 0.005, 'adam_beta1': 0.9, 'warmup_steps': 4}, run_summary=None)


In [50]:
base.reset_seed()

In [51]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [52]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [53]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [54]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

In [55]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-embedd",
    n_trials=150
)

[I 2025-03-15 13:59:12,992] A new study created in memory with name: Distill-embedd


Trial 0 with params: {'learning_rate': 0.0001025350969016849, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 2, 'lambda_param': 0.1, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.389400,2.202489,0.176902,0.003538,0.020000,0.006012
2,2.130400,2.065815,0.176902,0.003538,0.020000,0.006012
3,2.056500,1.989177,0.180568,0.006305,0.021096,0.007710
4,1.931500,1.870954,0.312557,0.035147,0.056676,0.035178
5,1.833500,1.757392,0.360220,0.039369,0.069720,0.044458
6,1.709600,1.661590,0.404216,0.041021,0.085248,0.054684
7,1.626300,1.575806,0.414299,0.063855,0.088030,0.057975
8,1.549100,1.512766,0.430797,0.081987,0.094931,0.067834
9,1.489000,1.455442,0.443630,0.072119,0.100824,0.075004
10,1.435200,1.417142,0.468378,0.089458,0.111196,0.084425


[I 2025-03-15 14:00:49,585] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 1.4347159517201392e-05, 'weight_decay': 0.009000000000000001, 'adam_beta1': 0.96, 'warmup_steps': 3, 'lambda_param': 0.0, 'temperature': 7.0}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.730600,1.332466,0.496792,0.111809,0.137176,0.115544
2,1.109200,1.012473,0.633364,0.240945,0.251073,0.233405
3,0.741900,0.790973,0.707608,0.352872,0.350676,0.337280
4,0.472700,0.678512,0.750687,0.474042,0.422351,0.427781
5,0.306200,0.630988,0.774519,0.526285,0.493095,0.491358
6,0.215200,0.618997,0.772686,0.569703,0.521625,0.527311
7,0.152300,0.604175,0.785518,0.639156,0.577941,0.589856
8,0.116600,0.590123,0.794684,0.680377,0.647333,0.650796
9,0.097900,0.572954,0.800183,0.685154,0.656027,0.660779
10,0.086600,0.576300,0.801100,0.700695,0.668766,0.672652


[I 2025-03-15 18:17:44,329] Trial 148 finished with value: 0.7198241340232989 and parameters: {'learning_rate': 0.0035743722892889324, 'weight_decay': 0.001, 'adam_beta1': 0.91, 'warmup_steps': 0, 'lambda_param': 0.2, 'temperature': 2.5}. Best is trial 126 with value: 0.7476729369943443.


Trial 149 with params: {'learning_rate': 0.002718471657061312, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 1, 'lambda_param': 0.5, 'temperature': 3.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.836000,1.399695,0.463795,0.104342,0.118371,0.099805
2,1.176500,1.035076,0.621448,0.221994,0.235325,0.214955
3,0.797500,0.810713,0.707608,0.348186,0.341596,0.326221
4,0.524300,0.703849,0.747938,0.460183,0.429056,0.423754
5,0.353500,0.647583,0.760770,0.480148,0.458097,0.455168
6,0.246600,0.642005,0.769019,0.553778,0.488316,0.496891
7,0.183700,0.617373,0.782768,0.621850,0.552141,0.571648
8,0.147300,0.606639,0.784601,0.623171,0.575613,0.581030
9,0.114100,0.594457,0.793767,0.672992,0.628306,0.634259
10,0.097600,0.592116,0.797434,0.671314,0.643691,0.645155


[I 2025-03-15 18:20:23,302] Trial 149 finished with value: 0.7096346115125874 and parameters: {'learning_rate': 0.002718471657061312, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 1, 'lambda_param': 0.5, 'temperature': 3.5}. Best is trial 126 with value: 0.7476729369943443.


In [56]:
print(best_trial2)

BestRun(run_id='126', objective=0.7476729369943443, hyperparameters={'learning_rate': 0.004849961480952609, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 4, 'lambda_param': 0.8, 'temperature': 5.0}, run_summary=None)


In [57]:
base.reset_seed()

In [58]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_fine_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_fine_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [59]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [60]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [61]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [62]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

In [63]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug-embedd",
    n_trials=150
)

[I 2025-03-15 18:20:23,677] A new study created in memory with name: Base-aug-embedd


Trial 0 with params: {'learning_rate': 0.0001025350969016849, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 32}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.470500,1.834866,0.534372,0.143789,0.150608,0.128960
2,1.332600,1.349108,0.659028,0.299349,0.275610,0.264669
3,0.829900,1.151894,0.707608,0.423168,0.382134,0.382695
4,0.535300,1.106892,0.736939,0.485646,0.463364,0.460583
5,0.340700,1.123481,0.749771,0.519467,0.512656,0.503252


[I 2025-03-15 18:21:18,040] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 2.6368755339723032e-05, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 46}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.735800,1.311783,0.801100,0.763316,0.698510,0.711612
2,0.031800,1.386834,0.791934,0.707166,0.712729,0.693158
3,0.012400,1.540610,0.804766,0.684653,0.716482,0.688974
4,0.007700,1.768018,0.796517,0.707599,0.712018,0.690358
5,0.008900,1.791564,0.795600,0.697440,0.728842,0.700632
6,0.006600,1.864163,0.792851,0.744543,0.714526,0.716044
7,0.005400,2.026941,0.798350,0.755110,0.731943,0.728429
8,0.006000,2.066484,0.802933,0.760251,0.733201,0.731364
9,0.004800,2.083822,0.791934,0.755811,0.729415,0.727348
10,0.002900,2.246291,0.804766,0.750409,0.751272,0.737517


[I 2025-03-16 08:29:14,725] Trial 148 finished with value: 0.7488210755640284 and parameters: {'learning_rate': 0.0026416561240707064, 'weight_decay': 0.003, 'adam_beta1': 0.97, 'warmup_steps': 19}. Best is trial 121 with value: 0.7876336386869527.


Trial 149 with params: {'learning_rate': 0.004436340314329022, 'weight_decay': 0.0, 'adam_beta1': 0.92, 'warmup_steps': 32}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.545600,1.189200,0.807516,0.792285,0.709570,0.733787
2,0.026300,1.429172,0.819432,0.793994,0.744090,0.753338
3,0.015300,1.427815,0.813016,0.787785,0.756419,0.758791
4,0.015300,1.845165,0.801100,0.788107,0.738296,0.741752
5,0.020500,1.870116,0.795600,0.722478,0.734648,0.711950
6,0.009500,2.117030,0.803850,0.719812,0.737215,0.709728
7,0.011800,2.219728,0.802016,0.764559,0.745803,0.734383
8,0.010400,2.157964,0.802933,0.780042,0.746955,0.746188
9,0.006700,2.298450,0.805683,0.753484,0.741116,0.733583
10,0.005300,2.438079,0.802016,0.786067,0.746044,0.752318


[I 2025-03-16 08:38:23,870] Trial 149 finished with value: 0.7660287672962912 and parameters: {'learning_rate': 0.004436340314329022, 'weight_decay': 0.0, 'adam_beta1': 0.92, 'warmup_steps': 32}. Best is trial 121 with value: 0.7876336386869527.


In [64]:
print(best_trial3)

BestRun(run_id='121', objective=0.7876336386869527, hyperparameters={'learning_rate': 0.004220661238506302, 'weight_decay': 0.005, 'adam_beta1': 0.92, 'warmup_steps': 51}, run_summary=None)


In [65]:
base.reset_seed()

In [66]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_fine_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_fine_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [67]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [68]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [69]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

In [70]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug-embedd",
    n_trials=150
)

[I 2025-03-16 08:38:24,130] A new study created in memory with name: Distill-aug-embedd


Trial 0 with params: {'learning_rate': 0.0001025350969016849, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 32, 'lambda_param': 0.1, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.614900,1.233279,0.532539,0.131392,0.149685,0.126736
2,0.945900,0.939816,0.651696,0.274430,0.248557,0.232239
3,0.659300,0.802266,0.706691,0.351731,0.321257,0.313599
4,0.491000,0.740946,0.732356,0.403767,0.367625,0.367174
5,0.375400,0.692331,0.757104,0.506122,0.433462,0.448497
6,0.296500,0.656675,0.769936,0.538537,0.476443,0.488199
7,0.241900,0.649463,0.776352,0.531836,0.499089,0.505106
8,0.202700,0.635340,0.776352,0.544576,0.512211,0.515118
9,0.175400,0.624493,0.784601,0.630730,0.558692,0.578574
10,0.153400,0.615558,0.792851,0.646697,0.593026,0.609056


[I 2025-03-16 08:41:37,248] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 1.4347159517201392e-05, 'weight_decay': 0.009000000000000001, 'adam_beta1': 0.96, 'warmup_steps': 38, 'lambda_param': 0.0, 'temperature': 7.0}


...

[I 2025-03-16 19:18:53,664] Trial 106 finished with value: 0.7889678575446838 and parameters: {'learning_rate': 0.000957289736022682, 'weight_decay': 0.01, 'adam_beta1': 0.93, 'warmup_steps': 28, 'lambda_param': 0.2, 'temperature': 5.0}. Best is trial 51 with value: 0.8121965030181337.


Trial 107 with params: {'learning_rate': 0.0016029320835677379, 'weight_decay': 0.006, 'adam_beta1': 0.91, 'warmup_steps': 36, 'lambda_param': 0.0, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss


[W 2025-03-16 19:18:59,784] Trial 107 failed with parameters: {'learning_rate': 0.0016029320835677379, 'weight_decay': 0.006, 'adam_beta1': 0.91, 'warmup_steps': 36, 'lambda_param': 0.0, 'temperature': 4.0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2548, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", li

KeyboardInterrupt: 

In [71]:
print(best_trial4)

NameError: name 'best_trial4' is not defined

In [72]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)

Best normal training score:  BestRun(run_id='52', objective=0.7267722601596618, hyperparameters={'learning_rate': 0.004185238693319757, 'weight_decay': 0.005, 'adam_beta1': 0.9, 'warmup_steps': 4}, run_summary=None)
Best distilation trianing score:  BestRun(run_id='126', objective=0.7476729369943443, hyperparameters={'learning_rate': 0.004849961480952609, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 4, 'lambda_param': 0.8, 'temperature': 5.0}, run_summary=None)
Best normal training score with augmentations:  BestRun(run_id='121', objective=0.7876336386869527, hyperparameters={'learning_rate': 0.004220661238506302, 'weight_decay': 0.005, 'adam_beta1': 0.92, 'warmup_steps': 51}, run_summary=None)


NameError: name 'best_trial4' is not defined